<a href="https://colab.research.google.com/github/teteusmds/teteusmds/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1)

In [1]:
import numpy as np

class Perceptron:
    def __init__(self, input_size, epochs=1000, learning_rate=0.01):
        self.weights = np.zeros(input_size)
        self.bias = 0
        self.epochs = epochs
        self.learning_rate = learning_rate

    def predict(self, inputs):
        linear_output = np.dot(inputs, self.weights) + self.bias
        return 1 if linear_output > 0 else 0

    def train(self, training_inputs, labels):
        for _ in range(self.epochs):
            for inputs, label in zip(training_inputs, labels):
                prediction = self.predict(inputs)
                update = self.learning_rate * (label - prediction)
                self.weights += update * inputs
                self.bias += update

# Dados de treinamento para o operador XOR
training_inputs = []
training_inputs.append(np.array([1, 1]))
training_inputs.append(np.array([1, 0]))
training_inputs.append(np.array([0, 1]))
training_inputs.append(np.array([0, 0]))

labels = np.array([0, 1, 1, 0])  # Tabela verdade XOR

# Inicializando e treinando o Perceptron
perceptron = Perceptron(2)
perceptron.train(training_inputs, labels)

# Testando para [1,1] e [0,1]
inputs1 = np.array([1, 1])
p1 = perceptron.predict(inputs1)
print(f"Output para [1,1]: {p1}")

inputs2 = np.array([0, 1])
p2 = perceptron.predict(inputs2)
print(f"Output para [0,1]: {p2}")


Output para [1,1]: 1
Output para [0,1]: 0


# Resultados:
# Saída para [1,1]: O Perceptron simples retorna sempre 0 para este caso, porque ele não consegue capturar a relação não linear necessária.

# Saída para [0,1]: O Perceptron pode retornar 1 ou outro valor inconsistente com a lógica XOR, dependendo do peso inicial e do treinamento.



#2)

# O resultado do Perceptron deu errado na tentativa de resolver o operador XOR devido a uma limitação fundamental do modelo

 # O problema do XOR não é linearmente separável
 # Para resolver o XOR com um modelo linear, é preciso transformar o problema. Uma rede neural multicamada, por exemplo, cria combinações não lineares dos dados para que eles se tornem separáveis em um espaço maior.

# 3)

In [2]:
import numpy as np

class Perceptron_to_Adaline(object):

    def __init__(self, no_of_inputs, threshold=0.5, nIterations=100, learning_rate=0.01):
        self.nIterations = nIterations
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.weights = np.zeros(no_of_inputs)
        self.bias = 0  # Adicionando um termo de bias para o Adaline

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def predict(self, inputs):
        summation = np.dot(inputs, self.weights) + self.bias
        activation = self.sigmoid(summation)
        return activation

    def train(self, training_inputs, labels):
          for _ in range(self.nIterations):
            for inputs, label in zip(training_inputs, labels):

                prediction = self.predict(inputs)

                error = label - prediction
                delta = self.learning_rate * error * inputs

                self.weights += delta
                self.bias += self.learning_rate * error


In [3]:
# Dados de treinamento para o XOR
training_inputs = [
    np.array([1, 1]),
    np.array([1, 0]),
    np.array([0, 1]),
    np.array([0, 0])
]
labels = np.array([0, 1, 1, 0])


adaline = Perceptron_to_Adaline(no_of_inputs=2, nIterations=10000, learning_rate=0.1)


adaline.train(training_inputs, labels)

test_inputs = [np.array([1, 1]), np.array([0, 1])]
for test in test_inputs:
    prediction = adaline.predict(test)
    print(f"Entrada: {test}, Previsão: {prediction}")


Entrada: [1 1], Previsão: 0.5128176319164417
Entrada: [0 1], Previsão: 0.5


# 4)

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def load_data():
    url = '/content/diabetes.csv'
    df = pd.read_csv(url)
    data = df[df.columns[:-1]]
    normalized_data = (data - data.min()) / (data.max() - data.min())

    labels = df[df.columns[-1]]
    X_train, X_test, y_train, y_test = train_test_split(normalized_data, labels, test_size=0.2, random_state=0)
    return X_train, X_test, y_train, y_test

class Perceptron:
    def __init__(self, n_iter=100, learning_rate=0.01, threshold=0.5):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.weights = None

    def train(self, X, y):
        self.weights = np.zeros(X.shape[1] + 1)
        for _ in range(self.n_iter):
            for xi, target in zip(X, y):
                update = self.learning_rate * (target - self.predict(xi))

                self.weights[1:] += update * xi
                self.weights[0] += update
        return self.weights

    def predict(self, X):
        activation = np.dot(X, self.weights[1:]) + self.weights[0]
        return 1 if activation >= self.threshold else 0

class Adaline:
    def __init__(self, n_iter=100, learning_rate=0.01, threshold=0.5):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.threshold = threshold
        self.weights = None

    def train(self, X, y):
        self.weights = np.zeros(X.shape[1] + 1)
        for _ in range(self.n_iter):
            for xi, target in zip(X, y):

                prediction = self.predict(xi)
                error = target - prediction
                self.weights[1:] += self.learning_rate * error * xi
                self.weights[0] += self.learning_rate * error
        return self.weights

    def predict(self, X):
        activation = np.dot(X, self.weights[1:]) + self.weights[0]
        return 1 if activation >= self.threshold else 0

def evaluate_model(model, X_train, X_test, y_train, y_test, learning_rate, threshold):
    model.learning_rate = learning_rate
    model.threshold = threshold
    model.train(X_train.values, y_train.values)

    correct_predictions = 0
    for xi, target in zip(X_test.values, y_test.values):
        prediction = model.predict(xi)
        if prediction == target:
            correct_predictions += 1

    accuracy = correct_predictions / len(y_test) * 100
    return accuracy

training_inputs, test_inputs, training_labels, test_labels = load_data()

perceptron_results = {}
adaline_results = {}

configurations = [
    (0.01, 0.2),
    (0.1, 0.2),
    (0.01, 0.5),
    (0.1, 0.5)
]

for learning_rate, threshold in configurations:
    perceptron = Perceptron(learning_rate=learning_rate, threshold=threshold)
    accuracy = evaluate_model(perceptron, training_inputs, test_inputs, training_labels, test_labels, learning_rate, threshold)
    perceptron_results[(learning_rate, threshold)] = accuracy

for learning_rate, threshold in configurations:
    adaline = Adaline(learning_rate=learning_rate, threshold=threshold)
    accuracy = evaluate_model(adaline, training_inputs, test_inputs, training_labels, test_labels, learning_rate, threshold)
    adaline_results[(learning_rate, threshold)] = accuracy

# Exibe os resultados
print("Taxa de acerto para o Perceptron:")
for config, accuracy in perceptron_results.items():
    print(f"Taxa de Aprendizado = {config[0]}, Limiar = {config[1]}: {accuracy:.2f}%")

print("\nTaxa de acerto para o Adaline:")
for config, accuracy in adaline_results.items():
    print(f"Taxa de Aprendizado = {config[0]}, Limiar = {config[1]}: {accuracy:.2f}%")


Taxa de acerto para o Perceptron:
Taxa de Aprendizado = 0.01, Limiar = 0.2: 70.78%
Taxa de Aprendizado = 0.1, Limiar = 0.2: 70.78%
Taxa de Aprendizado = 0.01, Limiar = 0.5: 71.43%
Taxa de Aprendizado = 0.1, Limiar = 0.5: 70.78%

Taxa de acerto para o Adaline:
Taxa de Aprendizado = 0.01, Limiar = 0.2: 70.78%
Taxa de Aprendizado = 0.1, Limiar = 0.2: 70.78%
Taxa de Aprendizado = 0.01, Limiar = 0.5: 71.43%
Taxa de Aprendizado = 0.1, Limiar = 0.5: 70.78%
